# JustWatch 작품 크롤링

In [ ]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm_notebook
import re
import copy
import random
import os
from bs4 import BeautifulSoup

- 위에 자기가 import한게 없으면 추가할것!

## 01. JustWatch(kr) 사이트에서 '드라마 한국 제목와 기본정보' 크롤링

In [ ]:
# 홈페이지를 selenium으로 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm&exclude_genres=ani,doc,rly&release_year_until=2023')


# 팝업창 닫기
try:
    driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
except NoSuchElementException:
    pass

time.sleep(1)

# 현재 페이지 높이 가져오기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 콘텐츠가 로드되기를 기다림
    time.sleep(2)
    
    # 새로운 페이지 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 높이가 이전 높이와 같으면, 더 이상 로드할 콘텐츠가 없다는 의미
    if new_height == last_height:
        break
    
    last_height = new_height

# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

driver.close()

# url_list 만들기
base_url = 'https://www.justwatch.com'
url_list = []

# 타이틀 리스트 내의 URL을 추출
for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0]) + 1)):
    tmp = soup.select_one('#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child(' + str(i) + ') > a')
    if tmp:
        tmp_url = tmp.get('href')
        if tmp_url:
            url = base_url + tmp_url
            url_list.append(url)

In [ ]:
# 각 드라마 데이터
drama_list = []

for x in tqdm_notebook(url_list):
    driver = webdriver.Chrome()
    driver.get(x)
    
    # 팝업창 닫기
    try:
        driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
    except NoSuchElementException:
        pass

    time.sleep(1.5)

    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 'detail-infos' 클래스의 모든 'div' 태그를 찾습니다.
    detail_infos = soup.find_all('div', class_='detail-infos')

    # 중복을 제거하기 위해 집합(set)을 사용합니다.
    unique_detail_infos = list({str(info): info for info in detail_infos}.values())

    # 정보를 저장할 변수 초기화
    play_time_div = None
    genre_div = None
    rating_div = None
    country_div = None

    # 필요한 정보를 가진 'div' 태그를 각각 찾습니다.
    for info in unique_detail_infos:
        heading = info.find('h3', class_='detail-infos__subheading')
        if heading:
            if '시간' in heading.text:
                play_time_div = info.find('div', class_='detail-infos__value')
            elif '장르' in heading.text:
                genre_div = info.find('div', class_='detail-infos__value')
            elif '연령' in heading.text:
                rating_div = info.find('div', class_='detail-infos__value')
            elif 'Production' in heading.text:
                country_div = info.find('div', class_='detail-infos__value')

    # 정보를 텍스트로 가져옵니다.
    play_time = play_time_div.text.strip() if play_time_div else None
    genre = genre_div.text.strip() if genre_div and genre_div.text.strip() else None
    rating = rating_div.text.strip() if rating_div else None
    country = country_div.text.strip() if country_div else None

    # 추가 정보 추출
    title_block = soup.find('div', class_='title-block')
    korean_title_element = title_block.find('h1') if title_block and title_block.find('h1') else None
    year_element = title_block.find('span',class_='text-muted') if title_block and title_block.find('span') else None
    original_title_element = title_block.find('h3') if title_block and title_block.find('h3') else None
    seasons_element = soup.find('h2', class_='subheading')

    korean_title = korean_title_element.text.strip() if korean_title_element else None
    year = year_element.text[2:-2].strip() if year_element else None
    original_title = original_title_element.text[5:].strip() if original_title_element else None
    seasons = seasons_element.text[:-5].strip() if seasons_element else None

    # 드라마 데이터 저장
    movie_data = {
        "title": korean_title,
        "original_title": original_title,
        "year": year,
        "season_episode": seasons,
        "runtime": play_time,
        "genre": genre,
        "age_rating": rating,
        "production_country": country
    }
    
    drama_list.append(movie_data)

# DataFrame 생성
jw_df = pd.DataFrame(drama_list)

# 드라이버 종료
driver.quit()

### Miss Data: 하위 코드는 크롤링 중 놓친 데이터 다시 수집

In [ ]:
miss_url_list = [url_list[i] for i in range(len(jw_df)) if jw_df.iloc[i]["title"] is None]
len(miss_url_list)

# 각 빠진 영화 데이터
miss_movie_list = []

for x in tqdm_notebook(miss_url_list):
    driver = webdriver.Chrome()
    driver.get(x)
    
    # 팝업창 닫기
    try:
        driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
    except NoSuchElementException:
        pass

    time.sleep(1.5)

    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 'detail-infos' 클래스의 모든 'div' 태그를 찾습니다.
    detail_infos = soup.find_all('div', class_='detail-infos')

    # 중복을 제거하기 위해 집합(set)을 사용합니다.
    unique_detail_infos = list({str(info): info for info in detail_infos}.values())

    # 정보를 저장할 변수 초기화
    play_time_div = None
    genre_div = None
    rating_div = None
    country_div = None

    # 필요한 정보를 가진 'div' 태그를 각각 찾습니다.
    for info in unique_detail_infos:
        heading = info.find('h3', class_='detail-infos__subheading')
        if heading:
            if '시간' in heading.text:
                play_time_div = info.find('div', class_='detail-infos__value')
            elif '장르' in heading.text:
                genre_div = info.find('div', class_='detail-infos__value')
            elif '연령' in heading.text:
                rating_div = info.find('div', class_='detail-infos__value')
            elif 'Production' in heading.text:
                country_div = info.find('div', class_='detail-infos__value')

    # 정보를 텍스트로 가져옵니다.
    play_time = play_time_div.text.strip() if play_time_div else None
    genre = genre_div.text.strip() if genre_div and genre_div.text.strip() else None
    rating = rating_div.text.strip() if rating_div else None
    country = country_div.text.strip() if country_div else None

    # 추가 정보 추출
    title_block = soup.find('div', class_='title-block')
    korean_title_element = title_block.find('h1') if title_block and title_block.find('h1') else None
    year_element = title_block.find('span',class_='text-muted') if title_block and title_block.find('span') else None
    original_title_element = title_block.find('h3') if title_block and title_block.find('h3') else None
    seasons_element = soup.find('h2', class_='subheading')

    korean_title = korean_title_element.text.strip() if korean_title_element else None
    year = year_element.text[2:-2].strip() if year_element else None
    original_title = original_title_element.text[5:].strip() if original_title_element else None
    seasons = seasons_element.text[:-5].strip() if seasons_element else None

    # 영화 데이터 저장
    movie_data = {
        "title": korean_title,
        "original_title": original_title,
        "year": year,
        "season_episode": seasons,
        "runtime": play_time,
        "genre": genre,
        "age_rating": rating,
        "Production country": country
    }
    
    miss_movie_list.append(movie_data)

# DataFrame 생성
miss_df = pd.DataFrame(miss_movie_list)

# 드라이버 종료
driver.quit()

In [ ]:
# 병합
justWatch = jw_df[jw_df['title'].notnull()]
justWatch = pd.concat([justWatch, miss_df])
justWatch.reset_index(drop=True,inplace=True)

In [ ]:
# 저장

justWatch.to_excel('../data/justwatch_first.xlsx')

## 02. JustWatch us 사이트에서 영어 제목 크롤링

In [ ]:
# 프록시 주소를 중간중간 변경하여 로봇에 탐지되는 것을 회피하기 위한 코드
# 시스템 프록시 설정 읽기 (예: Windows 환경 변수 사용)

proxy = os.environ.get('http_proxy') or os.environ.get('https_proxy')

options = webdriver.ChromeOptions()
options.add_argument(f'--proxy-server={proxy}')

driver = webdriver.Chrome(options=options)

In [ ]:
df = pd.read_excel('data/justwatch_first.xlsx', index_col=0)
df.head()

In [ ]:
countries_dict = {
    '대한민국': 'Korea',
    '벨기에': 'Belgium',
    '대만': 'Taiwan',
    '미국': 'US',
    '덴마크': 'Denmark',
    '독일': 'Germany',
    '일본': 'Japan',
    '태국': 'Thailand',
    '캐나다': 'Canada',
    '그리스': 'Greece',
    '영국': 'United Kingdom',
    'China': 'China',
    '스페인': 'Spain',
    '미국, 영국': 'US, United Kingdom',
    '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국': 'Iceland, Denmark, Finland, Sweden, Norway, Germany, France, United Kingdom',
    '미국, 프랑스': 'US, France',
    '호주': 'Australia',
    '스웨덴': 'Sweden',
    '이탈리아': 'Italy',
    '케냐': 'Kenya',
    'WEBAPP_COUNTRY_VN': 'Vietnam',
    '인도': 'India',
    '호주, 미국': 'Australia, US',
    '스위스': 'Switzerland',
    '필리핀 제도': 'Philippines',
    '영국, 미국': 'United Kingdom, US',
    '남아프리카공화국': 'South Africa',
    '폴란드': 'Poland',
    '브라질': 'Brazil',
    '미국, 캐나다': 'US, Canada',
    '콜롬비아': 'Colombia',
    '터키': 'Turkey',
    '룩셈부르크': 'Luxembourg',
    '핀란드': 'Finland',
    '이스라엘': 'Israel',
    '프랑스': 'France',
    '멕시코': 'Mexico',
    '벨기에, 네덜란드': 'Belgium, Netherlands',
    '덴마크, 노르웨이': 'Denmark, Norway',
    '노르웨이': 'Norway',
    '러시아': 'Russia',
    '미국, 일본': 'US, Japan',
    '영국, 프랑스': 'United Kingdom, France',
    '요르단': 'Jordan',
    '프랑스, 미국': 'France, US',
    '캐나다, 영국': 'Canada, United Kingdom',
    '태국, 미국': 'Thailand, US',
    '인도네시아': 'Indonesia',
    '사우디아라비아': 'Saudi Arabia',
    '영국, 인도': 'United Kingdom, India',
    'Arjantin': 'Argentina',
    '아이슬란드': 'Iceland',
    '오스트리아': 'Austria',
    '포르투갈': 'Portugal',
    '프랑스, 독일, 스페인, 영국': 'France, Germany, Spain, United Kingdom',
    '아일랜드, 영국': 'Ireland, United Kingdom',
    '캐나다, 프랑스, 독일': 'Canada, France, Germany',
    '이스라엘, 노르웨이': 'Israel, Norway',
    '대한민국, 미국': 'South Korea, US',
    '미국, 대한민국': 'US, South Korea',
    '네덜란드': 'Netherlands',
    '멕시코, 스페인': 'Mexico, Spain',
    '홍콩': 'Hong Kong',
    '칠레': 'Chile',
    '뉴질랜드, 호주': 'New Zealand, Australia',
    '나이지리아': 'Nigeria',
    '이집트': 'Egypt',
    '일본, 미국': 'Japan, US',
    '미국, 멕시코': 'US, Mexico',
    '쿠웨이트': 'Kuwait',
    '나이지리아, 미국': 'Nigeria, US',
    '말레이시아, 대만': 'Malaysia, Taiwan',
    '사우디아라비아, 인도': 'Saudi Arabia, India',
    '벨기에, 룩셈부르크': 'Belgium, Luxembourg',
    '사우디아라비아, 튀니지, 이집트, 모로코, 레바논': 'Saudi Arabia, Tunisia, Egypt, Morocco, Lebanon',
    '독일, 오스트리아': 'Germany, Austria',
    '체코 공화국, 독일': 'Czech Republic, Germany',
    '영국, 스페인': 'United Kingdom, Spain',
    '스페인, 브라질': 'Spain, Brazil',
    '레바논': 'Lebanon'
}

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.google.com/')

In [46]:
null_index = df[df['original_title'].isna()].index

In [ ]:
# 구글에서 justwatch 페이지 가져오기
english_title = []
original_title = []

for idx, row in tqdm_notebook(df.iterrows()):
    if idx not in null_index:
        title = row['original_title']
    else:
        title = row['title']
    year = row['year']
    country = countries_dict[row['Production country']]

    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(f'{title}({year}) {country} TV show justwatch')
    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.ENTER)

    time.sleep(1)

    if 'justwatch' in driver.find_element(By.CSS_SELECTOR, 'a[jsname=UWckNb]').get_attribute('href'):
        driver.find_element(By.CSS_SELECTOR, 'div > div > span > a > h3').click()
    else:
        original_title.append('검색 결과 부정확')
        english_title.append('검색 결과 부정확')
        driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()
        continue
    
    time.sleep(1)

    # justwatch에서 'too many request' 오류가 날 경우를 대비해 except문 추가
    try:
        if year == int(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > span').text[1:5]):
            english_title.append(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > h1').text)
            try:
                original_title.append(driver.find_element(By.CSS_SELECTOR, 'div.title-block > h3').text.split(': ', 1)[1])
            except:
                original_title.append('오리지날 타이틀 없음')
        else:
            original_title.append('검색 결과 부정확')
            english_title.append('검색 결과 부정확')
    except:
        original_title.append('검색 결과 부정확')
        english_title.append('검색 결과 부정확')

    driver.back()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()

In [ ]:
df['add_original_title'] = original_title
df['add_english_title'] = english_title

### Miss Data: 'too many request'로 거부되어 누락된 정보 재입력

In [ ]:
# 'too many request'로 거부되어 누락된 정보 재입력

for idx, row in tqdm_notebook(df[df['add_english_title'] == '검색 결과 부정확'].iterrows()):
    if idx not in null_index:
        title = row['original_title']
    else:
        title = row['title']
    year = row['year']
    country = countries_dict[row['Production country']]

    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(f'{title}({year}) {country} TV show justwatch')
    driver.find_element(By.CSS_SELECTOR, '#APjFqb').send_keys(Keys.ENTER)

    time.sleep(1)

    if 'justwatch' in driver.find_element(By.CSS_SELECTOR, 'a[jsname=UWckNb]').get_attribute('href'):
        driver.find_element(By.CSS_SELECTOR, 'div > div > span > a > h3').click()
    else:
        df.loc[idx,'add_original_title'] = '검색 결과 부정확'
        df.loc[idx,'add_english_title'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()
        continue
    
    time.sleep(1)

    # justwatch에서 'too many request' 오류가 날 경우를 대비해 except문 추가
    try:
        if year == int(driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > span').text[1:5]):
            df.loc[idx,'add_english_title'] = driver.find_element(By.CSS_SELECTOR, 'div.title-block > div > h1').text
            try:
                df.loc[idx,'add_original_title'] = driver.find_element(By.CSS_SELECTOR, 'div.title-block > h3').text.split(': ', 1)[1]
            except:
                df.loc[idx,'add_original_title'] = '오리지날 타이틀 없음'
        else:
            df.loc[idx,'add_original_title'] = '검색 결과 부정확'
            df.loc[idx,'add_english_title'] = '검색 결과 부정확'
    except:
        df.loc[idx,'add_original_title'] = '검색 결과 부정확'
        df.loc[idx,'add_english_title'] = '검색 결과 부정확'

    driver.back()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg > path').click()

In [ ]:
# 저장
df.to_excel('add_eng_title.xlsx')

## 03. JustWatch us 사이트에서 연령 크롤링

In [ ]:
df = pd.read_excel('data/add_eng_title.xlsx', index_col=0)
df.head()

In [ ]:
# justwatch에서 직접 검색하여 크롤링 하기
# 데이터 프레임에 컬럼 준비
df['justwatch_us_age'] = None
# 검색 결과 첫번째만 확인

for idx, row in tqdm_notebook(df.iterrows()):
    # 데이터 프레임의 영문 제목과 작품의 공개 연도를 변수에 할당
    search_title, search_year = row['add_english_title'], row['year']
    time.sleep(2)

    # 검색창에 검색어 입력
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(f'{search_title}')
    # 검색 실시
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)
    time.sleep(2)

    # 첫번째 검색 결과물의 제목
    title = driver.find_element(By.CLASS_NAME, 'header-title').text
    # 첫번째 검색 결과물의 연도
    year = int(driver.find_element(By.CLASS_NAME, 'header-year').text[1:5])

    if search_title == title and search_year == year:
        # 첫번째 검색 결과물 클릭
        driver.find_element(By.CLASS_NAME, 'header-title').click()
    else:
        df.loc[idx, 'justwatch_us_age'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').click()
        driver.find_element(By.CSS_SELECTOR, 'ion-searchbar > div > button').click()
        continue
    
    time.sleep(2)

    # 연령 정보 추출
    drama_info1 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
    age_info_tf = False
    for elements1 in drama_info1:
        if len(elements1.text.split('\n')) > 1:
            info_name1, runtime_info = elements1.text.split('\n')[0], elements1.text.split('\n')[1]
            if info_name1 == 'RUNTIME' and runtime_info == row['runtime'].replace('시간', 'h').replace('분', 'min'):
                age_info_tf = True
                break
            df.loc[idx, 'justwatch_us_age'] = '런타임 다름'

    if age_info_tf:
        drama_info2 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
        for elements2 in drama_info2:
            if len(elements2.text.split('\n')) > 1:
                info_name2, age_info = elements2.text.split('\n')[0], elements2.text.split('\n')[1]
                if info_name2 == 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = age_info
                    break
                elif info_name2 != 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = '연령 정보 없음'
                    continue

    try:
        # 버튼 찾기 (예: id가 'submit_button'인 버튼)
        button = driver.find_element(By.CSS_SELECTOR, "ion-header > ion-toolbar > ion-buttons > ion-button").click()
    except:
        pass

    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.BACKSPACE * len(search_title))

### Miss Data: 누락된 정보 재입력

In [ ]:
driver.find_elements(By.CLASS_NAME, 'title-list-row__column-header')[2].get_attribute('href')

action = ActionChains(driver)

In [ ]:
# justwatch에서 직접 검색하여 크롤링 하기
# 검색 결과 전체 리스트 확인

for idx, row in tqdm_notebook(df[df['justwatch_us_age'] == '검색 결과 부정확'].iterrows()):
    # 데이터 프레임의 영문 제목과 작품의 공개 연도를 변수에 할당
    search_title, search_year = row['add_english_title'], row['year']
    time.sleep(2)

    # 검색창에 검색어 입력
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(f'{search_title}')
    # 검색 실시
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)
    time.sleep(3)

    # 첫번째 검색 결과물의 제목
    title = driver.find_element(By.CLASS_NAME, 'header-title').text
    # 첫번째 검색 결과물의 연도
    year = int(driver.find_element(By.CLASS_NAME, 'header-year').text[1:5])
    t = 0
    for item in driver.find_elements(By.CLASS_NAME, 'title-list-row__column-header'):
        if len(item.text.split('(')) < 2:
            t += 1
            continue
        else:
            title = item.text.split('(')[0]
            year = int(item.text.split('(')[1][:-1])
            if search_title == title and search_year == year:
                driver.get(item.get_attribute('href'))
                break
    if search_title != title or search_year != year:
        df.loc[idx, 'justwatch_us_age'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').click()
        driver.find_element(By.CSS_SELECTOR, 'ion-searchbar > div > button').click()
        continue
    
    time.sleep(2)

    # 연령 정보 추출
    drama_info1 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
    age_info_tf = False
    for elements1 in drama_info1:
        if len(elements1.text.split('\n')) > 1:
            info_name1, runtime_info = elements1.text.split('\n')[0], elements1.text.split('\n')[1]
            if info_name1 == 'RUNTIME' and runtime_info == row['runtime'].replace('시간', 'h').replace('분', 'min'):
                age_info_tf = True
                break
            df.loc[idx, 'justwatch_us_age'] = '런타임 다름'

    if age_info_tf:
        drama_info2 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
        for elements2 in drama_info2:
            if len(elements2.text.split('\n')) > 1:
                info_name2, age_info = elements2.text.split('\n')[0], elements2.text.split('\n')[1]
                if info_name2 == 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = age_info
                    break
                elif info_name2 != 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = '연령 정보 없음'
                    continue

    try:
        # 버튼 찾기 (예: id가 'submit_button'인 버튼)
        button = driver.find_element(By.CSS_SELECTOR, "ion-header > ion-toolbar > ion-buttons > ion-button").click()
    except:
        pass

    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.BACKSPACE * len(search_title))

In [ ]:
# 저장
df.to_excel('justwatch_us_age.xlsx')

## 04. JustWatch 사이트에서 '좋아요, 싫어요' 크롤링

- 결론: JustWatch에서 드라마 제목, 영어 제목, ....을 사용